In [ ]:
!pip install rdkit-pypi

In [ ]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
import numpy as np


In [ ]:
dataset = pd.read_csv('1-unique_clean_smiles.csv')
dataset.shape

(17650, 3)

In [37]:
dataset.head()

,codes,smiles,values
0,1,O=C(O)P(=O)(O)O,1
1,2,NCc1cn(-c2ccccc2)nc1C12CC3CC(CC(C3)C1)C2,1
2,3,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,1
3,4,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...,1
4,5,CN[C@@H]1[C@@H](O)C(OC2[C@@H](N)C[C@@H](N)[C@H...,1


## Calculate descriptors using RDkit

### a. General molecular descriptors-about 200 molecular descriptors

In [42]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(dataset['smiles'])

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:1761: RuntimeWarning: overflow encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:1761: RuntimeWarning: invalid value encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)


In [43]:
df_with_200_descriptorss = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_200_descriptorss

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.467731,-4.487461,10.467731,1.696759,0.424095,126.004,122.980,125.971810,42,0,...,0,0,0,0,0,0,0,0,0,0
1,9.163315,-4.433800,9.163315,0.008726,0.937072,307.441,282.241,307.204848,120,0,...,0,0,0,0,0,0,0,0,0,0
2,16.630408,-8.173020,16.630408,2.011766,0.237873,733.937,666.401,733.461241,298,0,...,0,0,0,0,0,0,0,0,0,0
3,16.153679,-4.142295,16.153679,0.117580,0.730424,389.387,369.227,389.149932,148,0,...,0,0,0,0,0,0,0,0,0,0
4,9.531285,-5.872222,9.531285,1.499618,0.164624,496.558,456.238,496.274443,200,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17645,15.019732,-7.159199,15.019732,1.253429,0.608135,500.617,468.361,500.198108,188,0,...,0,1,0,0,0,0,0,0,0,0
17646,15.019732,-7.159199,15.019732,1.253429,0.608135,500.617,468.361,500.198108,188,0,...,0,1,0,0,0,0,0,0,0,0
17647,15.117218,-7.325142,15.117218,1.378554,0.593363,506.665,468.361,506.245058,194,0,...,0,1,0,0,0,0,0,0,0,0
17648,15.117218,-7.325142,15.117218,1.378554,0.593363,506.665,468.361,506.245058,194,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
# Reset index of unique_smiles to avoid issues during merging.
unique_smiles = dataset.reset_index(drop=True)
df_merged = pd.concat([unique_smiles, df_with_200_descriptorss], axis=1)

# Display the merged dataframe
print(df_merged.tail())

In [47]:
df_merged.to_csv('/content/2-2D descriptors RDKIT.csv', index=False)